In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

Setting environment for Local Macbook Pro


Cannot import mxnet.
xgboost is not loaded.

# Based on contour annotation

In [2]:
windowing_settings

{1: {'patch_size': 224, 'spacing': 56},
 2: {'comment': 'larger margin', 'patch_size': 224, 'spacing': 56},
 3: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 32},
 4: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 128},
 5: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 64},
 6: {'comment': 'twice as large patch', 'patch_size': 448, 'spacing': 64},
 7: {'comment': 'specify size/spacing in terms of microns rather than pixels',
  'patch_size_um': 103.04,
  'spacing_um': 30},
 8: {'comment': 'larger patch', 'patch_size_um': 206.08, 'spacing_um': 30},
 9: {'comment': 'larger patch', 'patch_size_um': 412.16, 'spacing_um': 30},
 10: {'comment': 'larger patch', 'patch_size_um': 824.32, 'spacing_um': 30}}

In [ ]:
win_id = 7
for stack in ['MD585', 'MD594', 'MD589']:
    grid_index_class_lookup, latest_timestamp = generate_annotation_to_grid_indices_lookup(stack, win_id=win_id, by_human=True, 
                                                                         suffix='contours', timestamp='latest',
                                                                         surround_margins=[200, 500],
                                                                        return_timestamp=True)
    
    grid_index_class_lookup_fp = \
    DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, 
                                                               by_human=True, timestamp=latest_timestamp)
    print grid_index_class_lookup_fp

    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
# win_id = 3
# windowing_properties = windowing_settings[win_id]
# patch_size = windowing_properties['patch_size']
# spacing = windowing_properties['spacing']
# w, h = metadata_cache['image_shape'][stack]
# half_size = patch_size/2
# grid_spec = (patch_size, spacing, w, h)

# Based on aligned atlas

In [ ]:
positive_level = {}
for name_u in all_known_structures:
    if name_u == '4N':
        positive_level[name_u] = 0.6
    elif name_u == '10N':
        positive_level[name_u] = 0.7
    else:
        positive_level[name_u] = 0.8
        
# positive_level = 0.8

In [ ]:
from learning_utilities import *

In [ ]:
# stack = 'ChatCryoJane201710'
for stack in [
 'MD652',
 'MD653',
 'MD657',
'MD658']:
    win_id = 5
    detector_id = 13
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                                    detector_id_f=detector_id,
                                                                        surround_margins=[200],
                                                                        positive_level=0.8, negative_level=0.1)
    grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')
    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
# stack = 'ChatCryoJane201710'
for stack in [
 'MD661',
 'MD662']:
    win_id = 7
    detector_id = 1
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                                    detector_id_f=detector_id,
                                                                        surround_margins=[200],
                                                                        positive_level=positive_level, 
                                                                         negative_level=0.1)
    grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')
    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

# Use human-corrected structure annotation file

In [2]:
positive_level = {}
for name_s in all_known_structures_sided:
    name_u = convert_to_unsided_label(name_s)
    if name_u == '4N':
        positive_level[name_s] = 0.6
    elif name_u == '10N':
        positive_level[name_s] = 0.7
    else:
        positive_level[name_s] = 0.8
        
# positive_level = 0.8

negative_level = .1

In [3]:
stack = 'CHATM2'

In [4]:
structures_df = DataManager.load_annotation_v4(stack=stack, by_human=True, suffix='structures', timestamp='04262018105643')

In [5]:
warped_volumes, warped_volumes_resolution = \
convert_structure_annotation_to_volume_origin_dict_v2(structures_df=structures_df, 
                                                      out_resolution='10.0um', stack=stack)

In [6]:
def get_structure_contours_from_structure_volumes_v2(volumes, stack, sections, 
                                                     resolution, level, sample_every=1,
                                                    use_unsided_name_as_key=False):
    """
    Re-section atlas volumes and obtain structure contours on each section.
    Resolution of output contours are in volume resolution.

    Args:
        volumes (dict of (3D array, 3-tuple)): {structure: (volume, origin_wrt_wholebrain)}. volume is a 3d array of probability values.
        sections (list of int):
        resolution (int): resolution of input volumes.
        level (float or dict): the cut-off probability at which surfaces are generated from probabilistic volumes. Default is 0.5.
        sample_every (int): how sparse to sample contour vertices.

    Returns:
        Dict {section: {name_s: contour vertices}}.
    """

    from collections import defaultdict
    
    structure_contours_wrt_alignedBrainstemCrop_rawResol = defaultdict(dict)

    converter = CoordinatesConverter(stack='CHATM2', section_list=metadata_cache['sections_to_filenames'][stack].keys())

    for name_s, (v, o) in volumes.iteritems():
        converter.derive_three_view_frames(name_s, 
                                           origin_wrt_wholebrain_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * o,
                                           zdim_um=convert_resolution_string_to_um(resolution=resolution, stack=stack) * v.shape[2])
        
    converter.register_new_resolution('structure_volume', resol_um=convert_resolution_string_to_um(resolution=resolution, stack=stack))
    converter.register_new_resolution('image', resol_um=convert_resolution_string_to_um(resolution='raw', stack=stack))
    
    for name_s, (structure_volume_volResol, origin_wrt_wholebrain_volResol) in volumes.iteritems():

        positions_of_all_sections_wrt_structureVolume = converter.convert_frame_and_resolution(
        p=np.array(sections)[:,None],
        in_wrt=('wholebrain', 'sagittal'), in_resolution='section',
        out_wrt=(name_s, 'sagittal'), out_resolution='structure_volume')[..., 2].flatten()

        structure_ddim = structure_volume_volResol.shape[2]
        
        valid_mask = (positions_of_all_sections_wrt_structureVolume >= 0) & (positions_of_all_sections_wrt_structureVolume < structure_ddim)
        if np.count_nonzero(valid_mask) == 0:
#             sys.stderr.write("valid_mask is empty.\n")
            continue

        positions_of_all_sections_wrt_structureVolume = positions_of_all_sections_wrt_structureVolume[valid_mask]
        positions_of_all_sections_wrt_structureVolume = np.round(positions_of_all_sections_wrt_structureVolume).astype(np.int)
        
        if isinstance(level, dict):
            level_this_structure = level[name_s]
        elif isinstance(level, float):
            level_this_structure = level

        contour_2d_wrt_structureVolume_sectionPositions_volResol = \
        find_contour_points_3d(structure_volume_volResol >= level,
                                along_direction='sagittal',
                                sample_every=sample_every,
                                positions=positions_of_all_sections_wrt_structureVolume)
        
        for d_wrt_structureVolume, cnt_uv_wrt_structureVolume in contour_2d_wrt_structureVolume_sectionPositions_volResol.iteritems():
            
            contour_3d_wrt_structureVolume_volResol = np.column_stack([cnt_uv_wrt_structureVolume, np.ones((len(cnt_uv_wrt_structureVolume),)) * d_wrt_structureVolume])
            
#             contour_3d_wrt_wholebrain_uv_rawResol_section = converter.convert_frame_and_resolution(
#                 p=contour_3d_wrt_structureVolume_volResol,
#                 in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
#                 out_wrt=('wholebrain', 'sagittal'), out_resolution='image_image_section')

            contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section = converter.convert_frame_and_resolution(
                p=contour_3d_wrt_structureVolume_volResol,
                in_wrt=(name_s, 'sagittal'), in_resolution='structure_volume',
                out_wrt=('wholebrainXYcropped', 'sagittal'), out_resolution='image_image_section')
            
            assert len(np.unique(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[:,2])) == 1
            sec = int(contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[0,2])
        
            if use_unsided_name_as_key:
                name = convert_to_unsided_label(name_s)
            else:
                name = name_s
            structure_contours_wrt_alignedBrainstemCrop_rawResol[sec][name] = contour_3d_wrt_alignedBrainstemCrop_uv_rawResol_section[..., :2]

    return structure_contours_wrt_alignedBrainstemCrop_rawResol

In [8]:
sample_every = 5

In [9]:
structure_contours_wrt_alignedBrainstemCrop_rawResol_pos_level = \
get_structure_contours_from_structure_volumes_v2(warped_volumes, stack=stack,
#                                                  sections=metadata_cache['valid_sections'][stack],
                                                 sections=[102],
                                                 resolution=warped_volumes_resolution,
                                                 level=positive_level,
                                                 sample_every=sample_every, 
                                                 use_unsided_name_as_key=True)

structure_contours_wrt_alignedBrainstemCrop_rawResol_neg_level = \
get_structure_contours_from_structure_volumes_v2(warped_volumes, stack=stack,
#                                                  sections=metadata_cache['valid_sections'][stack],
                                                 sections=[102],
                                                 resolution=warped_volumes_resolution,
                                                 level=negative_level,
                                                 sample_every=sample_every, 
                                                  use_unsided_name_as_key=True)

In [10]:
grid_spec = win_id_to_gridspec(win_id=7, stack=stack)
surround_margins_um = [200, 500]

In [11]:
patch_indices_allSections_allStructures = {}

sections = set(structure_contours_wrt_alignedBrainstemCrop_rawResol_pos_level.keys()) | \
set(structure_contours_wrt_alignedBrainstemCrop_rawResol_neg_level.keys())

for sec in sections:

    sys.stderr.write('Computing grid indices lookup for section %d...\n' % sec)

    mask_tb = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec, prep_id='alignedBrainstemCrop')

    patch_indices_thisSection_allStructures = {}

    if sec in structure_contours_wrt_alignedBrainstemCrop_rawResol_pos_level:
        
        patch_indices_thisSection_allStructures.update(
        locate_patches_v2(grid_spec=grid_spec, 
                          mask_tb=mask_tb,
                          polygons=structure_contours_wrt_alignedBrainstemCrop_rawResol_pos_level[sec].items(), \
                          surround_margins=surround_margins_um, 
                          categories=['positive']))

    if sec in structure_contours_wrt_alignedBrainstemCrop_rawResol_neg_level:

        patch_indices_thisSection_allStructures.update(
        locate_patches_v2(grid_spec=grid_spec, 
                          mask_tb=mask_tb,
                          polygons=structure_contours_wrt_alignedBrainstemCrop_rawResol_neg_level[sec].items(), \
                          surround_margins=surround_margins_um, 
                          categories=['surround']))

    if len(patch_indices_thisSection_allStructures) > 0:
        patch_indices_allSections_allStructures[sec] = patch_indices_thisSection_allStructures

Computing grid indices lookup for section 102...


In [ ]:
if return_timestamp:
    return DataFrame(patch_indices_allSections_allStructures).T, timestamp
else:
    return DataFrame(patch_indices_allSections_allStructures).T

In [13]:
for stack in ['CHATM2']:
    
    win_id = 7
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup_v2(stack=stack, win_id=win_id, 
                                                                            by_human=True, 
                                                                        surround_margins=[200, 500],
                                                                        positive_level=positive_level, 
                                                                         negative_level=0.1,
                                                                        suffix='structures',
                                                                           timestamp='04262018105643')
#     grid_index_class_lookup_fp = \
# DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
#                                                            timestamp='latest',
#                                                           suffix='structures')
    
#     save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
#     upload_to_s3(grid_index_class_lookup_fp)

TypeError: get_structure_contours_from_structure_volumes() got an unexpected keyword argument 'first_sec'

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
                                                       timestamp='02062018184326',
                                                      suffix='structures')

save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
%load_ext autoreload
%autoreload 2

from data_manager import *

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
                                                           timestamp='02062018184326',
                                                          suffix='structures')

In [ ]:
grid_index_class_lookup_fp

# Examine

In [ ]:
%matplotlib inline

In [ ]:
sec = 150

In [ ]:
# img = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2, version='gray')
img = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2)

In [ ]:
viz = img.copy()

# for s in ['VCA', 'VCP']:
for s in ['7N', '7N_surround_200_noclass', 
          '5N_surround_200_noclass',
         'Pn_surround_200_noclass']:
    all_locs = grid_parameters_to_sample_locations(win_id=win_id, stack=stack)
    if not sec in grid_index_class_lookup[s].dropna():
        sys.stderr.write("No %s patches on section %d.\n" % (s, sec))
        continue
    for x, y in all_locs[grid_index_class_lookup[s][sec]]:
        cv2.circle(viz, (x,y), 20, (255,0,0), 3)

In [ ]:
# viz = img.copy()

# # for s in ['VCA', 'VCP']:
# for s in ['7N_L', '7N_L_surround_200_noclass', 
#           '5N_L_surround_200_noclass',
#          'Pn_L_surround_200_noclass']:
#     all_locs = grid_parameters_to_sample_locations(win_id=win_id, stack=stack)
#     if not sec in grid_index_class_lookup[s].dropna():
#         sys.stderr.write("No %s patches on section %d.\n" % (s, sec))
#         continue
#     for x, y in all_locs[grid_index_class_lookup[s][sec]]:
#         cv2.circle(viz, (x,y), 20, (255,0,0), 3)

In [ ]:
display_image(viz[::4, ::4])

# Save and Upload

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, 
                                                           by_human=True, timestamp=latest_timestamp)
print grid_index_class_lookup_fp

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')

In [ ]:
save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
stack = 'MD595'

In [ ]:
grid_index_class_lookup = \
DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m=atlas_name, 
                                                                                detector_id_f=detector_id,
                                                           prep_id_f=2,
                                                                                warp_setting=warp_setting,
                                                          timestamp='latest')

In [ ]:
grid_index_class_lookup = DataManager.load_annotation_to_grid_indices_lookup(stack='MD589', win_id=5, 
                                                                             by_human=True,
                                                                            timestamp='latest')

In [ ]:
grid_index_class_lookup.loc[108]['VCA']

In [ ]:
# for stack in all_annotated_stacks:
    
#     sys.stderr.write('%s\n' % stack)
    
#     label_gridIndices = locate_annotated_patches_v2(stack=stack)
    
#     annotation_grid_indices_fn = os.path.join(ANNOTATION_ROOTDIR, stack, '%(stack)s_annotation_grid_indices.h5' % dict(stack=stack))
#     label_gridIndices.to_hdf(annotation_grid_indices_fn, 'grid_indices')

In [ ]:
# label_gridIndices = read_hdf(annotation_grid_indices_fn, 'grid_indices')